#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

FAISS is just a vector search library.

You feed it vectors.

It finds the most similar vectors efficiently.

You have to manually manage metadata.

ChromaDB is a vector database.

It stores both embeddings + associated metadata.

You can query with filtering (like SQL-like conditions).

Better for RAG pipelines where you need full control over the document store.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("larger_speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)
docs

[Document(metadata={'source': 'larger_speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'larger_speech.txt'}, page_content='…\n\nIt will be all the easier for us 

In [14]:
embeddings=OllamaEmbeddings(model="all-minilm:22m")
db=FAISS.from_documents(docs,embeddings)
db

so after storing all my data inputs as a vector in db i.e. fasis
then if i give it a query like prompt suppose for a text "india independence in 1947"
then i ask as "who got independce in 1947"
then this will be treated as query="who got independce in 1947"
and again query will be stored after embedding into vector in fasis and semantic search will be done

In [15]:
query="How does the speaker describe the desired outcome of the war?"
result_of=db.similarity_search(query)
result_of[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [16]:
print(f"Number of chunks created: {len(docs)}")

Number of chunks created: 5


#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers


k is nearest neighbours

In [20]:
retriever=db.as_retriever(search_kwargs={"k": 1})
docs=retriever.invoke(query)
docs[1].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [29]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
for doc in docs:
    print("\n the output is \n")
    print(doc.page_content)


 the output is 

It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.

 the output is 

The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance(manhattan distance). Therefore, a lower score is better.


In [30]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='11332ded-8fe6-4b54-bca5-576baa1d9687', metadata={'source': 'larger_speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(15.974321)),
 (Document(id='39969d30-6ee9-46f9-bc38-f62efef6b255', metadata={'source

till here we are giving queries but we can also give embedded vector of the query itself

In [31]:
embedded_query=embeddings.embed_query(query)
embedded_query

[-0.11221946030855179,
 0.3848111629486084,
 -0.002465207129716873,
 0.03732418641448021,
 -0.2030576765537262,
 0.3343668282032013,
 0.27403393387794495,
 -0.02678191475570202,
 0.00825434923171997,
 0.07606478035449982,
 -0.21836702525615692,
 0.054546236991882324,
 0.10464949905872345,
 -0.07646269351243973,
 -0.0987904742360115,
 0.05798334628343582,
 0.10039712488651276,
 -0.22238612174987793,
 0.018947608768939972,
 -0.0156472809612751,
 0.38640865683555603,
 0.320156455039978,
 0.3491297662258148,
 0.08060548454523087,
 0.21672499179840088,
 -0.047175247222185135,
 -0.08705932646989822,
 0.23342084884643555,
 -0.1535751223564148,
 0.20164808630943298,
 0.11621276289224625,
 -0.02108723297715187,
 0.10700991004705429,
 0.11959764361381531,
 0.08730784058570862,
 -0.06898330897092819,
 -0.165169358253479,
 -0.016831230372190475,
 -0.0052183158695697784,
 -0.11693349480628967,
 -0.24710366129875183,
 -0.22964419424533844,
 0.11679495126008987,
 0.15106041729450226,
 0.0792689025402

In [32]:
docs_and_score=db.similarity_search_with_score(embedded_query)
docs_and_score

[(Document(id='dd0929e0-b72b-499c-b63f-ef6b5383b655', metadata={'source': 'larger_speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will 

In [33]:
### Saving And Loading
db.save_local("faiss_index")

loading the faiss_index

In [35]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [36]:
docs

[Document(id='11332ded-8fe6-4b54-bca5-576baa1d9687', metadata={'source': 'larger_speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='39969d30-6ee9-46f9-bc38-f62efef6b255', metadata={'source': 'larger_speech.txt'}, pag